![](../images/rivacon_frontmark_combined_header.png)

# Example Notebook for Short Rate Models 

In [ ]:
import pyvacon.analytics as analytics
import datetime as dt
import pyvacon.tools.converter as converter
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.tools.enums as enums
import pyvacon.models.tools as model_tools
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib qt comment
#%matplotlib notebook

In [ ]:
refdate_p = dt.datetime(2017,2,20)
refdate = converter.getLTime(refdate_p)

### Create the necessary market data and setup models
For simplicity we just create one artificial discount curve using the analyticsTestData module and use this curve as fixing and discount curve.

In [ ]:
riskfreeCurve = mkt_testdata.InterestRate.Curves.EONIA(refdate_p) # get artificial test curve from th test market data module
cir = analytics.CIRModel('CIR_Test', refdate, 1.0,0.05,0.20,0.025)

### Create discount curves from the model

In [ ]:
dates =  [refdate_p+ dt.timedelta(days=1), refdate_p + dt.timedelta(days=30), refdate_p + dt.timedelta(days=60), refdate_p + dt.timedelta(days=180),
          refdate_p + dt.timedelta(days=365), refdate_p + dt.timedelta(days=2*365), refdate_p + dt.timedelta(days=3*365), 
          refdate_p + dt.timedelta(days=5*365)]
cir_dc = model_tools.compute_yieldcurve(cir, refdate,dates)
hull_white = analytics.HullWhiteModel('HW_Test', refdate, 1, 0.1, cir_dc)
hw_dc = model_tools.compute_yieldcurve(hull_white, refdate,dates)

In [ ]:
mkt_plot.curve(cir_dc, dates, refdate, True)#
mkt_plot.curve(hw_dc, dates, refdate, True)

### Model simulation
Tools for model analysis or simulation is provided by the ModelLab. In this section we show how one can simulate the CIR model for a given set of timepoints. On basis of this simulation, one may construct yieldcurves for the simulated values which can further be used to price products. With this approach one would be able to simulate whole portfolios at defined dates.

We first use the modellab to simulate the process underlying the model. The modellab simulates the short rate and keeps all simulated values in memory (for all paths).

In [ ]:
sim_times_p = [refdate_p + dt.timedelta(days=180)]
for i in range(10):
    sim_times_p.append(sim_times_p[-1] + dt.timedelta(days=(i+1)*180))
sim_times = converter.createPTimeList(refdate, sim_times_p)
n_sims = 100
max_num_threads = 2
cir_lab = analytics.ModelLab(cir, refdate)
cir_lab.simulate(sim_times, n_sims, 200, max_num_threads)
hw_lab = analytics.ModelLab(hull_white, refdate)
hw_lab.simulate(sim_times, n_sims, 200, max_num_threads)

Now that we have all simulated variables in memory, we can set the simulated values back into the model to compute simulated yieldcurves.

In [ ]:
sim_time_point = 10
sim_time_p = sim_times_p[sim_time_point]
sim_time = converter.getLTime(sim_time_p)
dc_dates_p = [1]
for j in range(20):
    dc_dates_p.append((j+1)*180)
dates = converter.createPTimeList(sim_time, dc_dates_p)

In [ ]:
for i in range(n_sims):
    cir_lab.setFromSimulatedValues(cir, 1, i)  
    dc = model_tools.compute_yieldcurve(cir, sim_times[sim_time_point], dc_dates_p)    
    mkt_plot.curve(dc, dates, sim_times[sim_time_point], True, '', False)
    

In [ ]:
for i in range(n_sims):
    hw_lab.setFromSimulatedValues(hull_white, 1, i)  
    dc = model_tools.compute_yieldcurve(hull_white, sim_times[sim_time_point], dc_dates_p)    
    mkt_plot.curve(dc, dates, sim_times[sim_time_point], True, '', False)

In [ ]:
time_slice = analytics.vectorDouble()
sim_time_point = 2
cir_lab.getTimeSlice(time_slice, sim_time_point, 0)
n_bins = 100
n, bins, patches = plt.hist(time_slice, n_bins, normed=1, facecolor='blue', alpha=0.75)